In [ ]:
# based on https://til.simonwillison.net/llms/python-react-pattern

In [1]:
import openai
import re
import httpx
import os
# from dotenv import load_dotenv

# _ = load_dotenv()
# from openai import OpenAI
from openai import OpenAI
client = OpenAI(base_url="http://10.2.125.37:1234/v1", api_key="lm-studio")

In [2]:
# chat_completion = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[{"role": "user", "content": "Hello world"}]
# )

chat_completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
  messages=[
    # {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Hello world"}
  ],
  temperature=0.7,
)

print(chat_completion.choices[0].message)

ChatCompletionMessage(content='It seems like you\'re trying to start with a classic "hello world" program, which is often used as a simple example in programming.\n\nHere\'s a basic Python code that prints "Hello World":\n\n```python\nprint("Hello World")\n```\n\nIf you want to create an executable file for this code (e.g., on Windows or macOS), here are the steps:\n\n1.  Open your preferred text editor or IDE.\n2.  Type in `print("Hello World")` and save it with a `.py` extension, e.g., `hello_world.py`.\n3.  On most operating systems, you can run this code by opening a terminal or command prompt, navigating to the directory where you saved the file, and typing `python hello_world.py`.\n\nHowever, if you\'re looking for something more interactive or want to create a graphical interface, there are many other libraries (e.g., Tkinter, PyQt) that allow you to do so.\n\nLet me know what you\'d like to achieve, and I can provide more tailored assistance!', role='assistant', function_call=N